In [48]:
import json
from collections import Counter
from tqdm import tqdm
import pickle
import numpy as np
import pandas as pd
pd.options.display.max_colwidth = 100
import matplotlib
matplotlib.use('Agg')
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import sys
import os
import argparse
from os.path import dirname, realpath

import hashlib
import datetime
import datetime as datetime
import sklearn
import statistics
sys.path.append("./src/")
#from disrisknet.utils.parsing import parse_args
import pdb

src_path = "G:\\FillmoreCancerData\\markhe\\VTERisk" 
src_path2 = "G:\\FillmoreCancerData\\markhe\\VTERisk - Copy" 

def md5(key):
    return hashlib.md5(repr(key).encode()).hexdigest()

from sklearn.metrics import confusion_matrix

In [49]:
testDF = pd.read_csv(os.path.join(src_path2, 'Notebooks/Find/fixed_dx.csv'))


In [50]:
#testDF = pd.read_csv(os.path.join(src_path2, 'Notebooks/Find/allChars_DF.csv'))
from sklearn.metrics import confusion_matrix
 

In [51]:
#logpath = os.path.join(src_path ,'logs_transformer_vte/y_CMP/10_17_Good/', "d8a1456b72625be735040911ffbe13c6.results.test_preds")
#logpath = os.path.join(src_path ,'logs_transformer_vte/F_Sen/', "8f35a8663b6409e1067edde7d3b57c07.results.test_preds")
logpath = os.path.join(src_path ,'logs_transformer_vte/y_CMP/', "d747b2da8d52da521474447204de0ba2.results.test_preds")


In [52]:
logpath = os.path.join(src_path ,'logs_transformer_vte/y_CMP/', "0948062a2982fcc14e697aae08268743.results.test_preds")
# frist to have varying TPs per patient with dropout

In [53]:
 
with open(logpath, 'rb') as f:
    R6 = pickle.load(f)
    p6 = np.array(R6['probs'])

pat_ids = (testDF['patient_id'] ).astype(int)
 

In [54]:
R6.keys()

dict_keys(['golds', 'probs', 'patient_golds', 'exams', 'pids', 'dates', 'censor_times', 'days_to_final_censors'])

In [55]:
testDF['pids'] = pat_ids.apply(md5)

g6 = pd.DataFrame({'p6': np.array(R6['probs'] )[:,1],    
                   'pids':np.array(R6['pids']),    
                   'censor_times':np.array(R6['censor_times']),    
                   'days_to_final_censors':np.array(R6['days_to_final_censors']), 
                    'golds':np.array(R6['golds']),    
                   'model_date':np.array(R6['dates'])
                  })
M = pd.merge(g6, testDF)

m1 = M[["patient_id", "index_date", "model_date",  "outcome", "golds","outcome_date","p6", "censor_times", "days_to_final_censors" ]]

In [56]:
p = p6[:,1]

In [57]:
R6.keys()

dict_keys(['golds', 'probs', 'patient_golds', 'exams', 'pids', 'dates', 'censor_times', 'days_to_final_censors'])

In [58]:
M.keys()

Index(['p6', 'pids', 'censor_times', 'days_to_final_censors', 'golds',
       'model_date', 'Unnamed: 0', 'patient_id', 'outcome', 'dob',
       'outcome_date', 'obs_time_end', 'index_date', 'dx_date',
       'obs_time_start', 'diag_date', 'Id_diag_diff'],
      dtype='object')

In [59]:
sklearn.metrics.roc_auc_score(m1['outcome'],  m1['p6'], average = 'samples')


0.5488859847718933

In [60]:
( confusion_matrix (m1['outcome'],  m1['p6']>.5) )

array([[5925,   39],
       [ 604,    3]], dtype=int64)

In [46]:
# how many people had event within 6 months of index date

In [47]:
m1.to_csv('output/yMulti1.csv') 

In [31]:
#m1.to_excel('output/dxMD_Sensitivity90.xlsx') 

In [35]:
statistics.quantiles( M['len180'],n=20)

KeyError: 'len180'

In [23]:
plt.hist(M['len180'])

(array([2.2715e+04, 1.8050e+03, 1.3000e+02, 2.5000e+01, 1.2000e+01,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 2.0000e+00]),
 array([1.00000e+00, 2.37810e+03, 4.75520e+03, 7.13230e+03, 9.50940e+03,
        1.18865e+04, 1.42636e+04, 1.66407e+04, 1.90178e+04, 2.13949e+04,
        2.37720e+04]),
 <BarContainer object of 10 artists>)

In [10]:
m['d']= pd.to_datetime(m['outcome_date']) - pd.to_datetime(m['model_date'])
m_VE = m[ m['outcome'] ]
m_VE1 = m_VE [   (pd.to_datetime( m_VE["outcome_date"] ) -pd.to_datetime( m_VE["index_date"] )) < datetime.timedelta(730)]
m_VE0 = m_VE [   (pd.to_datetime( m_VE["outcome_date"] ) -pd.to_datetime( m_VE["index_date"] )) >= datetime.timedelta(730)]
m_VE0["outcome"] = m_VE0["outcome"].replace(True, False)  
m0= m[ ~ m['outcome'] ]

m1 = pd.concat([m_VE1, m_VE0, m0])

C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_73432\1057840980.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m['d']= pd.to_datetime(m['outcome_date']) - pd.to_datetime(m['model_date'])
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_73432\1057840980.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_VE0["outcome"] = m_VE0["outcome"].replace(True, False)
